### Inverse Problems and Parameter Estimation, GEOS 627/427, University of Alaska Fairbanks

- script ex2p1_ex2p2.ipynb

In [ ]:
%matplotlib inline

In [ ]:
%run lib_header.py

In [ ]:
import scipy.io
import scipy.special
from numpy.linalg import inv

from lib_inverse import plot_ellipse
from lib_peip import chi2cdf
from lib_peip import chi2inv

In [ ]:
# load data
dat   = scipy.io.loadmat('data/data1.mat')
data1 = dat['data1']
print(f"data1: {data1.shape}")
ndata = len(data1)
t     = data1[:,0].reshape(ndata,1)
y     = data1[:,1].reshape(ndata,1)
sigma = data1[:,2].reshape(ndata,1)
ones  = np.ones((ndata,1))
ndata = len(t)

print('displaying t, y, sigma:')
showmat(np.hstack((t,y,sigma)),2)

In [ ]:
# Build the parabolic system matrix
G = np.hstack(((ones, t, -0.5*t**2)))
showmat(G)

In [ ]:
# Apply the weighting
yw = y/sigma
Gw = G / np.hstack((sigma,sigma,sigma))
showmat(Gw,2)

In [ ]:
# Solve for the least-squares solution
print('Least-squares solution, m =')
m = inv(Gw.T@Gw) @ Gw.T @ yw
showmat(m,1)

In [ ]:
ginv = inv(Gw.T@Gw) @ Gw.T
showmat(ginv,2)

In [ ]:
print('Covariance matrix')
covm = ginv @ ginv.T
showmat(covm,2)

In [ ]:
# see notes_tarantola.pdf
PCONF = 0.95
DELTA = scipy.special.erfinv(PCONF)*np.sqrt(2)
print(DELTA)

In [ ]:
# Get the 1.96-sigma (95%) conf intervals
print('%.2f%% parameter confidence intervals (m-, mest, m+)' % (PCONF*100))
delm = DELTA*np.sqrt(np.diag(covm)).reshape((3,1))
showmat(np.hstack((m-delm, m, m+delm)),2)

In [ ]:
# Because there are 3 parameters to estimate, we have ndata-3 degrees
# of freedom.
dof = ndata-3
print('Chi-square misfit for %.2f dof' %dof)
chi2 = np.linalg.norm((y.reshape((ndata, 1)) - G@m)/sigma.reshape((ndata, 1)))**2

# Find the p-value for this data set
print('chi-square p-value')
p = 1-chi2cdf(chi2,dof)

# Find the parameter correlations
s=np.sqrt(np.diag(covm)).reshape((3, 1))

print('correlation matrix')
r = covm / (s@s.T)

# Plot the data and model predicted data
xx = np.arange(np.min(t)-1,np.max(t)+1+0.05,0.05)
mm = m[0][0]+m[1][0]*xx-0.5*m[2][0]*xx**2

In [ ]:
plt.figure(figsize=(9,7))
plt.plot(xx,mm,'k')
#plt.plot(t,y,'o-')
# the flatten() commands are needed, unfortunately:
plt.errorbar(t.flatten(),y.flatten(),yerr=sigma.flatten(), ls='none',marker='o',mfc='none',capsize=3,mec='k',ms=3)
plt.xlabel('Time (s)')
plt.ylabel('Elevation (m)')
plt.show()

In [ ]:
print('Displaying Data and Model Fit (fig 1)')

# Output covm and the eigenvalues/eigenvectors of covm.
print('Covariance matrix for fitted parameters.')
print('Eigenvalues/eigenvectors of the covariance matrix')
lam,u = np.linalg.eig(inv(covm))
showmat([lam],2)
showmat(u,2)
print('%.1f%% confidence ellipsoid semiaxis lengths' % (PCONF*100))
semi_axes = np.sqrt(chi2inv(PCONF,3)*(1/lam))
showmat([semi_axes],2)
print('%.1f%% confidence ellipsoid semiaxes' % (PCONF*100))

In [ ]:
# Monte Carlo Section
y0 = G@m
nreal = 1000
mmc = np.zeros((3, nreal))
chimc = np.zeros(nreal)
Ginv = np.linalg.pinv(Gw)

for i in range(nreal):
    # Generate a trial data set of perturbed, weighted data
    noise = np.random.randn(ndata, 1) * sigma
    ytrial = y0 + noise
    ywtrial = ytrial / sigma
    mmx = Ginv @ ywtrial
    mmc[:, i] = mmx.flatten()
    chimc[i] = np.linalg.norm((G @ mmx - ytrial) / sigma) ** 2
#mmc = mmc.T

In [ ]:
# Plot the histogram of chi squared values
plt.figure()
plt.hist(chimc,30)
plt.ylabel('N')
plt.xlabel(r'$\chi^2$')
print('Displaying 1000 Monte-Carlo Chi-square Values (fig 2)')

In [ ]:
# Plot the histograms of the model parameters
plt.figure(figsize=(10,5))

plt.subplot(1,3,1)
plt.hist(mmc[0,:])
plt.title(r'$m_1 (m)$')

plt.subplot(1,3,2)
plt.hist(mmc[1,:])
plt.title(r'$m_2 (m/s)$')

plt.subplot(1,3,3)
plt.hist(mmc[2,:])
plt.title(r'$m_3 (m/s^2)$')
print('Displaying Monte-Carlo Model Histograms (fig 3)')

In [ ]:
# Plot the realizations of each pair of model parameters with the other
plt.figure(figsize=(10,5))

plt.subplot(1,3,1)
plt.plot(mmc[0,:],mmc[1,:],'k*')
plt.xlabel(r'$m_1 (m)$')
plt.ylabel(r'$m_2 (m/s)$')

plt.subplot(1,3,2)
plt.plot(mmc[0,:],mmc[2,:],'k*')
plt.xlabel(r'$m_1 (m)$')
plt.ylabel(r'$m_3 (m/s)$')

plt.subplot(1,3,3)
plt.plot(mmc[1,:],mmc[2,:],'k*')
plt.xlabel(r'$m_2 (m)$')
plt.ylabel(r'$m_3 (m/s)$')

print('Displaying Projections of 1000 Monte-Carlo models (fig 4)')

In [ ]:
# Plot the 95% error ellipses for each pair of parameters
# Note that because we're doing pairs of parameters there are 2
# degrees of freedom in the Chi-square here, rather than 3.  

deltachisq = chi2inv(PCONF,2)
delta = np.sqrt(deltachisq)

#generate a vector of angles from 0 to 2*pi
theta = np.arange(0,2*np.pi,0.01)
#the radii in each direction from the center
r = np.zeros((len(theta),2))

# compute the data for the m1, m2 ellipsoid.
indc = np.ix_([0,1],[0,1])
C = covm[indc]
#C = covm[:2,:2]
[lam,u] = np.linalg.eig(inv(C))
lam = np.diag(lam)
#calculate the x component of the ellipsoid for all angles
r[:,0] = (delta/np.sqrt(lam[0,0]))*u[0,0]*np.cos(theta)+(delta/np.sqrt(lam[1,1]))*u[0,1]*np.sin(theta)
#calculate the y component of the ellipsoid for all angles
r[:,1] = (delta/np.sqrt(lam[0,0]))*u[1,0]*np.cos(theta)+(delta/np.sqrt(lam[1,1]))*u[1,1]*np.sin(theta)

plt.figure(figsize=(15,6))
# plot the data for the m1, m2 ellipsoid
plt.subplot(1,3,1)
plt.plot(m[0]+r[:,0],m[1]+r[:,1],'k')
plt.fill(m[0]+r[:,0],m[1]+r[:,1],'k')
#axis([-50 50 85 110]);
plt.xlabel(r'$m_1 (m)$')
plt.ylabel(r'$m_2 (m/s)$')

# compute the data for the m1, m3 ellipsoid.
indc = np.ix_([0,2],[0,2])
C = covm[indc]
lam,u = np.linalg.eig(inv(C))
lam = np.diag(lam)

#calculate the x component of the ellipsoid for all angles
r[:,0] = (delta/np.sqrt(lam[0,0]))*u[0,0]*np.cos(theta)+(delta/np.sqrt(lam[1,1]))*u[0,1]*np.sin(theta)
#calculate the y component of the ellipsoid for all angles
r[:,1] = (delta/np.sqrt(lam[0,0]))*u[1,0]*np.cos(theta)+(delta/np.sqrt(lam[1,1]))*u[1,1]*np.sin(theta)
plt.subplot(1,3,2)
plt.plot(m[0]+r[:,0],m[2]+r[:,1],'k')
plt.fill(m[0]+r[:,0],m[2]+r[:,1],'k')
#axis([-50 50 85 110]);
plt.xlabel(r'$m_1 (m)$')
plt.ylabel(r'$m_3 (m/s)$')

# compute the data for the m2, m3 ellipsoid.
indc = np.ix_([1,2],[1,2])
C = covm[indc]
lam,u = np.linalg.eig(inv(C))
lam = np.diag(lam)
deltachisq = chi2inv(PCONF,2)
delta = np.sqrt(deltachisq)

#calculate the x component of the ellipsoid for all angles
r[:,0] = (delta/np.sqrt(lam[0,0]))*u[0,0]*np.cos(theta)+(delta/np.sqrt(lam[1,1]))*u[0,1]*np.sin(theta)
#calculate the y component of the ellipsoid for all angles
r[:,1] = (delta/np.sqrt(lam[0,0]))*u[1,0]*np.cos(theta)+(delta/np.sqrt(lam[1,1]))*u[1,1]*np.sin(theta)
plt.subplot(1,3,3)
plt.plot(m[1]+r[:,0],m[2]+r[:,1],'k')
plt.fill(m[1]+r[:,0],m[2]+r[:,1],'k')
#axis([-50 50 85 110]);
plt.xlabel(r'$m_2 (m)$')
plt.ylabel(r'$m_3 (m/s)$')

print('Displaying %.1f%% Confidence Ellipse Projections (fig 5)' % (PCONF*100))

In [ ]:
# SAME THING, but instead using plot_ellipse() from lib_geos.py

def plot_ellipse_custom(inds,DELTA2,C,m):
    indc = np.ix_(inds,inds)
    C = covm[indc]
    [x,y] = plot_ellipse(DELTA2,C,np.array([m[inds[0]],m[inds[1]]]))
    plt.plot(x,y,'r')

plt.figure(figsize=(15,6))

plt.subplot(1,3,1)
plot_ellipse_custom([0,1],deltachisq,C,m)
plt.xlabel(r'$m_1 (m)$')
plt.ylabel(r'$m_2 (m/s)$')

plt.subplot(1,3,2)
plot_ellipse_custom([0,2],deltachisq,C,m)
plt.xlabel(r'$m_1 (m)$')
plt.ylabel(r'$m_3 (m/s)$')

plt.subplot(1,3,3)
plot_ellipse_custom([1,2],deltachisq,C,m)
plt.xlabel(r'$m_2 (m)$')
plt.ylabel(r'$m_3 (m/s)$')
plt.show()